# Building a Company Financials Dataset in Python (S&P 500)

In this notebook I walk you step by step through how I build a
structured dataset of S&P 500 company financials using Python:

1. Fetch the S&P 500 constituents table from Wikipedia using **requests**
   and **BeautifulSoup** (to avoid 403 errors).
2. Use **yfinance** to talk to Yahoo Finance.
3. Pull basic information about each company (name, country, industry, etc.).
4. Pull the last 3 years of annual financial statements.
5. Combine everything into one tidy table (a pandas DataFrame).
6. Save the dataset as CSV and Excel.

## Step 0 – Install and import the needed packages 

- **pandas** – for working with tables (DataFrames).
- **yfinance** – for downloading data from Yahoo Finance.
- **requests** – for making HTTP requests.
- **BeautifulSoup** – for parsing the HTML from Wikipedia.


In [1]:
# !pip install pandas yfinance requests beautifulsoup4

In [2]:
#!pip install lxml

In [3]:
import time
import pandas as pd
import yfinance as yf
import requests
from bs4 import BeautifulSoup

## Step 1 – Get the S&P 500 list with requests + BeautifulSoup

In this step, the S&P 500 constituent list is fetched from Wikipedia.
On some networks, calling `pandas.read_html` directly on the URL
can give a **403 Forbidden** error, so I take a slightly more
manual approach:

1. The HTML of the page is downloaded using `requests`.
2. A browser-like **User-Agent** header is sent to header is sent to make the request appear                                                               more like it is from a normal browser and less like a bot.
3. The HTML is parsed using **BeautifulSoup** to find the table with the S&P 500 constituents.
4. The table HTML is passed to `pandas.read_html` to turn it into a
   DataFrame.

A couple of column names are standardised so they can be used later without concern about the exact 
naming on the page (for example, `Symbol` is renamed to `Ticker`).


In [4]:
def load_sp500_companies(max_companies=500):
    """ Function for loading the S&P 500 constituent table from Wikipedia.

      Parameters:
    ----------
    max_companies : int
        Maximum number of companies to keep. This gives me direct
        control over the "at least 100 and no more than 500" requirement.
    """
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

    # 1) Download the HTML
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/124.0 Safari/537.36"
        )
    }

    response = requests.get(url, headers=headers)
    response.raise_for_status()  # raises an error for 4xx/5xx status codes

    html = response.text

    # 2) Parse with BeautifulSoup
    soup = BeautifulSoup(html, "html.parser")

    # 3) Find the main S&P 500 table (id="constituents")
    table = soup.find("table", id="constituents")
    if table is None:
        # Fallback: first table on the page
        table = soup.find("table")
        if table is None:
            raise RuntimeError("Could not find any HTML table on the page")

    # 4) Convert the HTML table to a DataFrame
    df_list = pd.read_html(str(table))
    if not df_list:
        raise RuntimeError("pandas could not read the table HTML")

    sp500 = df_list[0]

    # Standardise column names for later use
    rename_map = {
        "Symbol": "Ticker",
        "Security": "Company",
    }
    sp500 = sp500.rename(columns=rename_map)

    # Limit the universe size
    sp500 = sp500.head(max_companies)
    return sp500


# I call the function once to see what the data looks like
sp500_df = load_sp500_companies(max_companies=150)
sp500_df.head()


C:\Users\User\AppData\Local\Temp\ipykernel_7332\2567963894.py:38: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_list = pd.read_html(str(table))


,Ticker,Company,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In the preview above you should now see a table with columns like
`Ticker`, `Company`, `GICS Sector`, `GICS Sub-Industry`, and
`Headquarters Location`.


## Step 2 – Get company metadata from Yahoo Finance

Now that the tickers have been obtained, let's know more about each company:

- Full company name
- Country of the main headquarters
- Industry and sector
- Reporting currency
- Number of employees
- Market capitalization

The `yfinance.Ticker` object is used, which provides a `.get_info()` method
that returns a dictionary with many details. This is wrapped in a small helper
function so it can be reused later.


In [5]:
def get_company_metadata(ticker_obj):
    """ Function to retrieve basic company metadata from a Yahoo Finance ticker object.

     This function safely queries Yahoo Finance for information about a given company.
    If the data request fails (e.g due to missing fields, connection issues, or API
    inconsistencies), the function returns an empty dictionary instead of raising an error,
    ensuring that failures for individual companies do not interrupt larger batch processes.
    """
    try:
        info = ticker_obj.get_info()
    except Exception:
        return {}

    return {
        "company_name": info.get("longName") or info.get("shortName") or ticker_obj.ticker,
        "country": info.get("country"),
        "industry": info.get("industry"),
        "sector": info.get("sector"),
        "currency": info.get("financialCurrency") or info.get("currency"),
        "employees": info.get("fullTimeEmployees"),
        "market_cap": info.get("marketCap"),
    }


# Quick demo for a single company (Apple)
demo_ticker = yf.Ticker("AAPL")
get_company_metadata(demo_ticker)


{'company_name': 'Apple Inc.',
 'country': 'United States',
 'industry': 'Consumer Electronics',
 'sector': 'Technology',
 'currency': 'USD',
 'employees': 166000,
 'market_cap': 4223723110400}

The output above is a Python dictionary with key–value pairs such as
`company_name`, `country`, and so on.


## Step 3 – Understand the financial statement data structure

For each company, Yahoo Finance exposes three main annual statements:

- `financials` – income statement (revenue, net income, etc.)
- `balance_sheet` – balance sheet (assets, liabilities, etc.)
- `cashflow` – cash flow statement (operating cash flow, etc.)

Each of these is a pandas DataFrame where:

- **Rows** are line items (for example, `Total Revenue`, `Net Income`).
- **Columns** are year-end dates (for example, `2023-12-31`).

To make life easier, a helper function called `safe_get_item` is written
that tries to pull one specific line for one date and returns
`None` if the data is missing.


In [6]:
def safe_get_item(df, row_label, date):
    """ Safely gets a single numeric value from a financial statement.

    - `df` is the DataFrame (income statement, balance sheet, or cashflow).
    - `row_label` is the line item I am interested in, such as
      'Total Revenue' or 'Net Income'.
    - `date` is the column (a specific period end).
    """
    if df is None or df.empty:
        return None
    if row_label not in df.index:
        return None
    if date not in df.columns:
        return None

    value = df.loc[row_label, date]
    return float(value) if pd.notna(value) else None


# Quick illustration with Apple again
income_demo = demo_ticker.financials
income_demo


,2025-09-30,2024-09-30,2023-09-30,2022-09-30,2021-09-30
Tax Effect Of Unusual Items,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN
Tax Rate For Calcs,1.560000e-01,2.410000e-01,1.470000e-01,1.620000e-01,NaN
Normalized EBITDA,1.447480e+11,1.346610e+11,1.258200e+11,1.305410e+11,NaN
Net Income From Continuing Operation Net Minority Interest,1.120100e+11,9.373600e+10,9.699500e+10,9.980300e+10,NaN
Reconciled Depreciation,1.169800e+10,1.144500e+10,1.151900e+10,1.110400e+10,NaN
Reconciled Cost Of Revenue,2.209600e+11,2.103520e+11,2.141370e+11,2.235460e+11,NaN
EBITDA,1.447480e+11,1.346610e+11,1.258200e+11,1.305410e+11,NaN
EBIT,1.330500e+11,1.232160e+11,1.143010e+11,1.194370e+11,NaN
Net Interest Income,NaN,NaN,-1.830000e+08,-1.060000e+08,1.980000e+08
Interest Expense,NaN,NaN,3.933000e+09,2.931000e+09,2.645000e+09


In the DataFrame above you can see the rows and columns that `safe_get_item`
will be working with.


## Step 4 – Pull the last 3 years of financials for one company

Now everything is put together for a **single ticker symbol**. The function
`extract_company_financials_for_last_years` will:

1. Create a `yfinance.Ticker` object.
2. Fetch metadata (name, country, industry, etc.).
3. Fetch the three financial statements.
4. Look at the column dates, sort them from newest to oldest, and keep
   the most recent 3.
5. For each of these dates, create one dictionary (one "row") with:
   - Ticker
   - Company name, country, industry, sector
   - Year
   - Revenue + revenue currency
   - Net income, gross profit, total assets, operating cash flow
   - Employees and market cap (these are current, not per year, but
     they are still useful KPIs).

The function returns a list of dictionaries, ready to be turned into
a DataFrame later.


In [7]:
def extract_company_financials_for_last_years(ticker_symbol, max_years=3):
    """ Collects up to `max_years` of annual financials for one company.

    This returns a list of dictionaries. Each dictionary represents
    one company–year combination.
    """
    ticker_obj = yf.Ticker(ticker_symbol)

    # 1) Basic metadata
    meta = get_company_metadata(ticker_obj)

    # 2) Financial statements
    try:
        income = ticker_obj.financials
        balance_sheet = ticker_obj.balance_sheet
        cashflow = ticker_obj.cashflow
    except Exception:
        return []

    if income is None or income.empty:
        return []

    all_dates = list(income.columns)
    if not all_dates:
        return []

    # Sort dates from newest to oldest and keep the first few
    sorted_dates = sorted(all_dates, reverse=True)
    selected_dates = sorted_dates[:max_years]

    rows = []

    for dt in selected_dates:
        year = pd.to_datetime(dt).year

        row = {
            "ticker": ticker_symbol,
            "company_name": meta.get("company_name"),
            "country": meta.get("country"),
            "industry": meta.get("industry"),
            "sector": meta.get("sector"),
            "year": int(year),
            # Revenue and its currency
            "revenue": safe_get_item(income, "Total Revenue", dt),
            "revenue_currency": meta.get("currency"),
            # Additional KPIs
            "net_income": safe_get_item(income, "Net Income", dt),
            "gross_profit": safe_get_item(income, "Gross Profit", dt),
            "total_assets": safe_get_item(balance_sheet, "Total Assets", dt),
            "operating_cash_flow": safe_get_item(
                cashflow,
                "Total Cash From Operating Activities",
                dt,
            ),
            # Snapshot KPIs
            "employees": meta.get("employees"),
            "market_cap": meta.get("market_cap"),
        }

        rows.append(row)

    return rows


# Test the function for one ticker
extract_company_financials_for_last_years("AAPL", max_years=3)


[{'ticker': 'AAPL',
  'company_name': 'Apple Inc.',
  'country': 'United States',
  'industry': 'Consumer Electronics',
  'sector': 'Technology',
  'year': 2025,
  'revenue': 416161000000.0,
  'revenue_currency': 'USD',
  'net_income': 112010000000.0,
  'gross_profit': 195201000000.0,
  'total_assets': 359241000000.0,
  'operating_cash_flow': None,
  'employees': 166000,
  'market_cap': 4223659671552},
 {'ticker': 'AAPL',
  'company_name': 'Apple Inc.',
  'country': 'United States',
  'industry': 'Consumer Electronics',
  'sector': 'Technology',
  'year': 2024,
  'revenue': 391035000000.0,
  'revenue_currency': 'USD',
  'net_income': 93736000000.0,
  'gross_profit': 180683000000.0,
  'total_assets': 364980000000.0,
  'operating_cash_flow': None,
  'employees': 166000,
  'market_cap': 4223659671552},
 {'ticker': 'AAPL',
  'company_name': 'Apple Inc.',
  'country': 'United States',
  'industry': 'Consumer Electronics',
  'sector': 'Technology',
  'year': 2023,
  'revenue': 383285000000.0

The result is a small list of dictionaries, each describing one year of
financials for Apple.


## Step 5 – Loop over many companies and build the full dataset

Now scalling up from one company to many is done by:

1. Looping over the first `max_companies` rows in the S&P 500 table.
2. For each ticker, `extract_company_financials_for_last_years` is called.
3. All the small lists are accumulated into one big list.
4. That list is converted into a single pandas DataFrame.
5. Rows where revenue is missing are optionally dropped to keep the dataset clean.

Additionally, a small `sleep_seconds` pause is added between requests so that the 
Yahoo Finance servers are not hammered too aggressively.


In [8]:
def build_financial_dataset_from_sp500(
    max_companies=150,
    max_years_per_company=3,
    sleep_seconds=0.5,
):
    """ This function creates the full multi-year dataset for a subset of S&P 500.

    Returns a pandas DataFrame with one row per company–year.
    """
    sp500 = load_sp500_companies(max_companies=max_companies)

    all_rows = []

    for i, row in sp500.iterrows():
        ticker_symbol = row["Ticker"]  # uses the renamed column
        print(f"[{i+1}/{len(sp500)}] Processing {ticker_symbol}...")

        try:
            company_rows = extract_company_financials_for_last_years(
                ticker_symbol=ticker_symbol,
                max_years=max_years_per_company,
            )
            all_rows.extend(company_rows)
        except Exception as e:
            print(f"  -> Skipping {ticker_symbol} due to error: {e}")

        time.sleep(sleep_seconds)

    dataset = pd.DataFrame(all_rows)

    # Optional cleaning step: keep only rows where revenue is present
    if "revenue" in dataset.columns:
        dataset = dataset[dataset["revenue"].notna()]

    return dataset


# This line actually builds the dataset. Depending on your internet
# connection it can take a while. You can lower max_companies for
# quicker experimentation.
financial_dataset = build_financial_dataset_from_sp500(
    max_companies=150,       # adjust between 100 and 500
    max_years_per_company=3,
    sleep_seconds=0.7,
)

financial_dataset.head(), financial_dataset.shape


C:\Users\User\AppData\Local\Temp\ipykernel_7332\2567963894.py:38: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_list = pd.read_html(str(table))


[1/150] Processing MMM...
[2/150] Processing AOS...
[3/150] Processing ABT...
[4/150] Processing ABBV...
[5/150] Processing ACN...
[6/150] Processing ADBE...
[7/150] Processing AMD...
[8/150] Processing AES...
[9/150] Processing AFL...
[10/150] Processing A...
[11/150] Processing APD...
[12/150] Processing ABNB...
[13/150] Processing AKAM...
[14/150] Processing ALB...
[15/150] Processing ARE...
[16/150] Processing ALGN...
[17/150] Processing ALLE...
[18/150] Processing LNT...
[19/150] Processing ALL...
[20/150] Processing GOOGL...
[21/150] Processing GOOG...
[22/150] Processing MO...
[23/150] Processing AMZN...
[24/150] Processing AMCR...
[25/150] Processing AEE...
[26/150] Processing AEP...
[27/150] Processing AXP...
[28/150] Processing AIG...
[29/150] Processing AMT...
[30/150] Processing AWK...
[31/150] Processing AMP...
[32/150] Processing AME...
[33/150] Processing AMGN...
[34/150] Processing APH...
[35/150] Processing ADI...
[36/150] Processing AON...
[37/150] Processing APA...
[

(  ticker             company_name        country  \
 0    MMM               3M Company  United States   
 1    MMM               3M Company  United States   
 2    MMM               3M Company  United States   
 3    AOS  A. O. Smith Corporation  United States   
 4    AOS  A. O. Smith Corporation  United States   
 
                          industry       sector  year       revenue  \
 0                   Conglomerates  Industrials  2024  2.457500e+10   
 1                   Conglomerates  Industrials  2023  2.461000e+10   
 2                   Conglomerates  Industrials  2022  3.422900e+10   
 3  Specialty Industrial Machinery  Industrials  2024  3.818100e+09   
 4  Specialty Industrial Machinery  Industrials  2023  3.852800e+09   
 
   revenue_currency    net_income  gross_profit  total_assets  \
 0              USD  4.173000e+09  1.012800e+10  3.986800e+10   
 1              USD -6.995000e+09  9.627000e+09  5.058000e+10   
 2              USD  5.777000e+09  1.499700e+10  4.645500

In [9]:
financial_dataset

,ticker,company_name,country,industry,sector,year,revenue,revenue_currency,net_income,gross_profit,total_assets,operating_cash_flow,employees,market_cap
0,MMM,3M Company,United States,Conglomerates,Industrials,2024,2.457500e+10,USD,4.173000e+09,1.012800e+10,3.986800e+10,None,61500.0,92160917504
1,MMM,3M Company,United States,Conglomerates,Industrials,2023,2.461000e+10,USD,-6.995000e+09,9.627000e+09,5.058000e+10,None,61500.0,92160917504
2,MMM,3M Company,United States,Conglomerates,Industrials,2022,3.422900e+10,USD,5.777000e+09,1.499700e+10,4.645500e+10,None,61500.0,92160917504
3,AOS,A. O. Smith Corporation,United States,Specialty Industrial Machinery,Industrials,2024,3.818100e+09,USD,5.336000e+08,1.456100e+09,3.240000e+09,None,12700.0,9394287616
4,AOS,A. O. Smith Corporation,United States,Specialty Industrial Machinery,Industrials,2023,3.852800e+09,USD,5.566000e+08,1.484800e+09,3.213900e+09,None,12700.0,9394287616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439,DLTR,"Dollar Tree, Inc.",United States,Discount Stores,Consumer Defensive,2024,1.678110e+10,USD,-9.984000e+08,6.019700e+09,2.202350e+10,None,64434.0,23782975488
440,DLTR,"Dollar Tree, Inc.",United States,Discount Stores,Consumer Defensive,2023,1.541150e+10,USD,1.615400e+09,5.781300e+09,2.302210e+10,None,64434.0,23782975488
441,D,"Dominion Energy, Inc.",United States,Utilities - Regulated Electric,Utilities,2024,1.445900e+10,USD,2.124000e+09,6.922000e+09,1.024150e+11,None,14700.0,51268956160
442,D,"Dominion Energy, Inc.",United States,Utilities - Regulated Electric,Utilities,2023,1.439300e+10,USD,2.031000e+09,6.958000e+09,1.090800e+11,None,14700.0,51268956160


In [10]:
# --- Scale money columns to billions for easier readability and rename them, drop operating_cash_flow ---

# 1) List of monetary columns I want in billions
money_cols = ["revenue", "net_income", "gross_profit", "total_assets", "market_cap"]

# 2) Convert from units (usually dollars) to billions
#    e.g. 10000000000 → 10.0
financial_dataset[money_cols] = financial_dataset[money_cols] / 1e9

# 3) Rename the columns to make it clear they are in billions of dollars
financial_dataset = financial_dataset.rename(columns={
    "revenue": "revenue_in_$B",
    "net_income": "net_income_in_$B",
    "gross_profit": "gross_profit_in_$B",
    "total_assets": "total_assets_in_$B",
    "market_cap": "market_cap_in_$B",
})

# 4) Drop operating_cash_flow entirely
financial_dataset = financial_dataset.drop(columns=["operating_cash_flow"])

financial_dataset["employees"] = (
    pd.to_numeric(financial_dataset["employees"], errors="coerce")
      .round(0)
      .astype("Int64")
)
# Quick check
financial_dataset.head()


,ticker,company_name,country,industry,sector,year,revenue_in_$B,revenue_currency,net_income_in_$B,gross_profit_in_$B,total_assets_in_$B,employees,market_cap_in_$B
0,MMM,3M Company,United States,Conglomerates,Industrials,2024,24.5750,USD,4.1730,10.1280,39.8680,61500,92.160918
1,MMM,3M Company,United States,Conglomerates,Industrials,2023,24.6100,USD,-6.9950,9.6270,50.5800,61500,92.160918
2,MMM,3M Company,United States,Conglomerates,Industrials,2022,34.2290,USD,5.7770,14.9970,46.4550,61500,92.160918
3,AOS,A. O. Smith Corporation,United States,Specialty Industrial Machinery,Industrials,2024,3.8181,USD,0.5336,1.4561,3.2400,12700,9.394288
4,AOS,A. O. Smith Corporation,United States,Specialty Industrial Machinery,Industrials,2023,3.8528,USD,0.5566,1.4848,3.2139,12700,9.394288


The tuple printed above shows:

- A preview of the first few rows.
- The overall shape `(number_of_rows, number_of_columns)`.

Each row represents a **company–year**, and the columns include:

- `ticker`, `company_name`, `country`, `industry`, `sector`
- `year`
- `revenue`, `revenue_currency`
- `net_income`, `gross_profit`, `total_assets`, `operating_cash_flow`
- `employees`, `market_cap`


# Validation :  Looking for missing values.

In [11]:
financial_dataset.isna().sum()

ticker                 0
company_name           0
country                0
industry               0
sector                 0
year                   0
revenue_in_$B          0
revenue_currency       0
net_income_in_$B       0
gross_profit_in_$B    54
total_assets_in_$B     0
employees              2
market_cap_in_$B       0
dtype: int64

In [12]:
# Missing gross profit rows:
financial_dataset[financial_dataset["gross_profit_in_$B"].isna()]

,ticker,company_name,country,industry,sector,year,revenue_in_$B,revenue_currency,net_income_in_$B,gross_profit_in_$B,total_assets_in_$B,employees,market_cap_in_$B
24,AFL,Aflac Incorporated,United States,Insurance - Life,Financial Services,2024,19.128000,USD,5.443000,NaN,117.566000,12694,58.344763
25,AFL,Aflac Incorporated,United States,Insurance - Life,Financial Services,2023,18.841000,USD,4.659000,NaN,126.724000,12694,58.344763
26,AFL,Aflac Incorporated,United States,Insurance - Life,Financial Services,2022,19.151000,USD,4.418000,NaN,131.738000,12694,58.344763
54,ALL,The Allstate Corporation,United States,Insurance - Property & Casualty,Financial Services,2024,63.515000,USD,4.667000,NaN,111.617000,55000,54.993560
55,ALL,The Allstate Corporation,United States,Insurance - Property & Casualty,Financial Services,2023,56.591000,USD,-0.188000,NaN,103.362000,55000,54.993560
56,ALL,The Allstate Corporation,United States,Insurance - Property & Casualty,Financial Services,2022,50.617000,USD,-1.289000,NaN,97.989000,55000,54.993560
78,AXP,American Express Company,United States,Credit Services,Financial Services,2024,65.949000,USD,10.129000,NaN,271.461000,75100,257.462518
79,AXP,American Express Company,United States,Credit Services,Financial Services,2023,60.515000,USD,8.374000,NaN,261.108000,75100,257.462518
80,AXP,American Express Company,United States,Credit Services,Financial Services,2022,52.862000,USD,7.514000,NaN,228.354000,75100,257.462518
81,AIG,"American International Group, Inc.",United States,Insurance - Diversified,Financial Services,2024,27.270000,USD,-1.404000,NaN,161.322000,22200,42.572427


In [13]:
# Fill gross_profit_in_$B with median *within each sector*
financial_dataset["gross_profit_in_$B"] = (
    financial_dataset
    .groupby("sector")["gross_profit_in_$B"]
    .transform(lambda s: s.fillna(s.median()))
)

# Fill employees with median *within each industry*
financial_dataset["employees"] = (
    financial_dataset
    .groupby("industry")["employees"]
    .transform(lambda s: s.fillna(s.median()))
    .round(0)
    .astype("Int64")
)

financial_dataset.isna().sum()


ticker                0
company_name          0
country               0
industry              0
sector                0
year                  0
revenue_in_$B         0
revenue_currency      0
net_income_in_$B      0
gross_profit_in_$B    0
total_assets_in_$B    0
employees             0
market_cap_in_$B      0
dtype: int64

This way:

* Missing gross profit for a tech company is filled using other tech companies.

* Missing employees for a pharma firm is filled using other pharma companies.

## Step 6 – Save the dataset to CSV and Excel

Finally, the dataset is saved so that it can be used outside of this
notebook (for example in Excel, Power BI, or another project).

Two files are written into the current working directory:

- `company_financials_long_format.csv`
- `company_financials_long_format.xlsx`


In [14]:
csv_path = "company_financials_long_format.csv"
xlsx_path = "company_financials_long_format.xlsx"

#financial_dataset.to_csv(csv_path, index=False)
#financial_dataset.to_excel(xlsx_path, index=False)

print("Saved CSV to:", csv_path)
print("Saved Excel to:", xlsx_path)


Saved CSV to: company_financials_long_format.csv
Saved Excel to: company_financials_long_format.xlsx


## Recap

In this notebook I have:

1. Loaded a list of S&P 500 companies using `requests` + BeautifulSoup.
2. Used `yfinance` to talk to Yahoo Finance.
3. Pulled metadata (company name, country, industry, sector, currency).
4. Pulled up to 3 years of annual financial statements for each company.
5. Combined everything into one tidy pandas DataFrame.
6. Saved the result as both CSV and Excel files.


